### Keras Tuner - Decide Number of Hidden Layers and Neuron in Neural Network

In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

<ipython-input-1-447605061030>:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
data = pd.read_csv('../ANN/Churn_Modelling.csv')

In [4]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
X = data.drop('Exited',axis=1)
y = data.Exited

In [6]:
# Creating dummy variables
geography = pd.get_dummies(X.Geography,drop_first=True)
gender = pd.get_dummies(X.Gender,drop_first=True)

In [7]:
# Concatenate the data frames
X = pd.concat([X,geography,gender],axis=1)

In [8]:
X = X.drop(['Geography','Gender'],axis=1)

In [9]:
X.drop('Surname',axis=1,inplace=True)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=0)

In [12]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [13]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Hyperparameters
1. How many number of hidden layers we should have?
2. How many number of neurons we should have in hidden layers?
3. Learning Rate 

In [14]:
 def build_model(hp):
     model = keras.Sequential()
     for i in range(hp.Int('num_layers',2,20)):
         model.add(layers.Dense(units=hp.Int('units_'+str(i),min_value = 3, max_value = 512, step = 32), activation='relu'))
     model.add(layers.Dense(1,activation='sigmoid'))
     model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate',values = [1e-2,1e-3,1e-4])),loss = 'binary_crossentropy',metrics = ['accuracy'])
     return model

In [15]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial = 3, 
    directory ='project1',
    project_name = 'Churn Modelling')

In [16]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 3, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 3, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [17]:
from sklearn.model_selection import train_test_split
X_train , X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 0)

In [18]:
tuner.search(X_train,y_train,epochs=10,validation_data = (X_test,y_test))

Trial 5 Complete [00h 01m 05s]
val_accuracy: 0.7929999828338623

Best val_accuracy So Far: 0.7929999828338623
Total elapsed time: 00h 05m 45s
INFO:tensorflow:Oracle triggered exit


In [19]:
tuner.results_summary(3)

Results summary
Results in project1\Churn Modelling
Showing 3 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 13
units_0: 67
units_1: 195
learning_rate: 0.0001
units_2: 3
units_3: 3
units_4: 3
units_5: 3
units_6: 3
units_7: 3
units_8: 3
units_9: 3
units_10: 3
units_11: 3
units_12: 3
Score: 0.7929999828338623
Trial summary
Hyperparameters:
num_layers: 8
units_0: 323
units_1: 35
learning_rate: 0.0001
units_2: 99
units_3: 483
units_4: 131
units_5: 291
units_6: 259
units_7: 483
units_8: 259
units_9: 3
units_10: 131
units_11: 419
units_12: 35
Score: 0.7929999828338623
Trial summary
Hyperparameters:
num_layers: 18
units_0: 3
units_1: 227
learning_rate: 0.001
units_2: 35
units_3: 3
units_4: 163
units_5: 163
units_6: 35
units_7: 419
units_8: 291
units_9: 195
units_10: 195
units_11: 451
units_12: 323
units_13: 3
units_14: 3
units_15: 3
units_16: 3
units_17: 3
Score: 0.7929999828338623


In [34]:
model = tuner.get_best_models(num_models=1)[0]

In [36]:
y_pred = model.predict_classes(X_test)

C:\Users\kc510\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
